In [1]:
'''
!pip install numpy
!pip install pandas
!pip install scipy
!pip install sklearn
!pip install matplotlib
'''

'\n!pip install numpy\n!pip install pandas\n!pip install scipy\n!pip install sklearn\n!pip install matplotlib\n'

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy.stats as stats
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [16]:
patients_raw = pd.read_csv('data/medical_clean.csv')
patients = patients_raw[['ReAdmis', 'HighBlood', 'Anxiety', 
                         'Stroke','Arthritis', 'Diabetes',
                         'Hyperlipidemia', 'BackPain', 'Allergic_rhinitis',
                         'Reflux_esophagitis', 'Asthma','Soft_drink', 'Overweight']].set_index('ReAdmis')
patients.head()

,HighBlood,Anxiety,Stroke,Arthritis,Diabetes,Hyperlipidemia,BackPain,Allergic_rhinitis,Reflux_esophagitis,Asthma,Soft_drink,Overweight
ReAdmis,,,,,,,,,,,,
No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,No,Yes,No,No
No,Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
No,Yes,No,No,No,Yes,No,No,No,No,No,No,Yes
No,No,No,Yes,Yes,No,No,No,No,Yes,Yes,No,No
No,No,No,No,No,No,Yes,No,Yes,No,No,Yes,No


In [15]:
readmitted = patients[patients['ReAdmis'] == 'Yes']
readmitted_count = readmitted.shape[0]
readmitted.head()

,HighBlood,Anxiety,Stroke,Arthritis,Diabetes,Hyperlipidemia,BackPain,Allergic_rhinitis,Reflux_esophagitis,Asthma,Soft_drink,Overweight
ReAdmis,,,,,,,,,,,,
Yes,Yes,No,No,No,No,Yes,No,No,No,No,No,No
Yes,No,No,No,No,No,Yes,No,Yes,No,Yes,No,No
Yes,No,No,No,No,No,Yes,No,No,Yes,No,No,Yes
Yes,No,No,No,No,No,Yes,Yes,No,No,No,No,Yes
Yes,Yes,No,No,Yes,No,No,No,Yes,Yes,No,No,No


In [5]:
not_readmitted = patients[patients['ReAdmis'] == 'No']
not_readmitted_count = not_readmitted.shape[0]
not_readmitted.head()

,ReAdmis,HighBlood,Anxiety,Stroke,Arthritis,Diabetes,Hyperlipidemia,BackPain,Allergic_rhinitis,Reflux_esophagitis,Asthma,Soft_drink,Overweight
0,No,Yes,Yes,No,Yes,Yes,No,Yes,Yes,No,Yes,No,No
1,No,Yes,No,No,No,No,No,No,No,Yes,No,No,Yes
2,No,Yes,No,No,No,Yes,No,No,No,No,No,No,Yes
3,No,No,No,Yes,Yes,No,No,No,No,Yes,Yes,No,No
4,No,No,No,No,No,No,Yes,No,Yes,No,No,Yes,No


In [59]:
readmit_df = pd.DataFrame({
    'HighBlood': [readmitted['HighBlood'].value_counts()['Yes'], not_readmitted['HighBlood'].value_counts()['Yes']],
    'Anxiety': [readmitted['Anxiety'].value_counts()['Yes'], not_readmitted['Anxiety'].value_counts()['Yes']],
    'Stroke': [readmitted['Stroke'].value_counts()['Yes'], not_readmitted['Stroke'].value_counts()['Yes']],
    'Arthritis': [readmitted['Arthritis'].value_counts()['Yes'], not_readmitted['Arthritis'].value_counts()['Yes']],
    'Diabetes': [readmitted['Diabetes'].value_counts()['Yes'], not_readmitted['Diabetes'].value_counts()['Yes']],
    'Hyperlipidemia': [readmitted['Hyperlipidemia'].value_counts()['Yes'], not_readmitted['Hyperlipidemia'].value_counts()['Yes']],
    'BackPain': [readmitted['BackPain'].value_counts()['Yes'], not_readmitted['BackPain'].value_counts()['Yes']],
    'Reflux_esophagitis': [readmitted['Reflux_esophagitis'].value_counts()['Yes'], not_readmitted['Reflux_esophagitis'].value_counts()['Yes']],
    'Asthma': [readmitted['Asthma'].value_counts()['Yes'], not_readmitted['Asthma'].value_counts()['Yes']],
    'Soft_drink': [readmitted['Soft_drink'].value_counts()['Yes'], not_readmitted['Soft_drink'].value_counts()['Yes']],
    'Overweight': [readmitted['Overweight'].value_counts()['Yes'], not_readmitted['Overweight'].value_counts()['Yes']],
    }, index=['Readmitted', 'Not_Readmitted'])
readmit_df

,HighBlood,Anxiety,Stroke,Arthritis,Diabetes,Hyperlipidemia,BackPain,Reflux_esophagitis,Asthma,Soft_drink,Overweight
Readmitted,4090,3215,1993,3574,2738,3372,4114,1530,2893,2575,7094
Not_Readmitted,2584,2030,1260,2245,1740,2125,2573,2605,1869,1614,4510


Null hypothesis is that all pre-existing conditions share the same readmission rate and that any variation can be attributed to chance.

In [79]:
row_mean = readmit_df.mean(axis=1)
ev = pd.DataFrame({
    'HighBlood': row_mean,
    'Anxiety': row_mean,
    'Stroke': row_mean,
    'Arthritis': row_mean,
    'Diabetes': row_mean,
    'Hyperlipidemia': row_mean,
    'Reflux_esophagitis': row_mean,
    'Asthma': row_mean,
    'Soft_drink': row_mean,
    'Overweight': row_mean    
})
ev

,HighBlood,Anxiety,Stroke,Arthritis,Diabetes,Hyperlipidemia,Reflux_esophagitis,Asthma,Soft_drink,Overweight
Readmitted,3380.727273,3380.727273,3380.727273,3380.727273,3380.727273,3380.727273,3380.727273,3380.727273,3380.727273,3380.727273
Not_Readmitted,2286.818182,2286.818182,2286.818182,2286.818182,2286.818182,2286.818182,2286.818182,2286.818182,2286.818182,2286.818182


In [81]:
row_sum = readmit_df.sum(axis=1)
re_sum = pd.DataFrame({
    'HighBlood': row_sum,
    'Anxiety': row_sum,
    'Stroke': row_sum,
    'Arthritis': row_sum,
    'Diabetes': row_sum,
    'Hyperlipidemia': row_sum,
    'Reflux_esophagitis': row_sum,
    'Asthma': row_sum,
    'Soft_drink': row_sum,
    'Overweight': row_sum    
})
re_sum

,HighBlood,Anxiety,Stroke,Arthritis,Diabetes,Hyperlipidemia,Reflux_esophagitis,Asthma,Soft_drink,Overweight
Readmitted,37188,37188,37188,37188,37188,37188,37188,37188,37188,37188
Not_Readmitted,25155,25155,25155,25155,25155,25155,25155,25155,25155,25155


In [74]:
box = [1] * 

0.15866666666666668